In [2]:
import os
import pandas as pd
import numpy as np
import time
import math
from sklearn.neural_network import MLPRegressor
from scipy import stats

In [3]:
def distSample(numbers, probabilities, rnd_num):
    # Sampling a single number from a discrete distribution
    #   The possible Numbers in the distribution with their resective
    #   Probabilities. rndNum is a randomly drawn probability
    #
    #   Conditions on Input (not checked):
    #   1. Numbers and Probabilites correspond one to one (i.e. first number is
    #   drawn w.p. first probability etc). These are numpy arrays.
    #   2. rndNum is a number between zero and one.
    #   3. Probabilites is a probability vector (numpy array)
    # The output is a number (float)

    cum_prob = 0
    sampled_int = 0
    while rnd_num > cum_prob:
        cum_prob += probabilities[sampled_int]
        sampled_int += 1

    return numbers[sampled_int - 1]


In [4]:
def CPC18_getDist(H, pH, L, lot_shape, lot_num):
    # Extract true full distributions of an option in CPC18
    #   input is high outcome (H: int), its probability (pH: double), low outcome
    #   (L: int), the shape of the lottery ('-'/'Symm'/'L-skew'/'R-skew' only), and
    #   the number of outcomes in the lottery (lot_num: int)
    #   output is a matrix (numpy matrix) with first column a list of outcomes (sorted
    #   ascending) and the second column their respective probabilities.

    if lot_shape == '-':
        if pH == 1:
            dist = np.array([H, pH])
            dist.shape = (1, 2)
        else:
            dist = np.array([[L, 1-pH], [H, pH]])

    else:  # H is multi outcome
        # compute H distribution
        high_dist = np.zeros(shape=(lot_num, 2))
        if lot_shape == 'Symm':
            k = lot_num - 1
            for i in range(0, lot_num):
                high_dist[i, 0] = H - k / 2 + i
                high_dist[i, 1] = pH * stats.binom.pmf(i, k, 0.5)

        elif (lot_shape == 'R-skew') or (lot_shape == 'L-skew'):
            if lot_shape == 'R-skew':
                c = -1 - lot_num
                dist_sign = 1
            else:
                c = 1 + lot_num
                dist_sign = -1
            for i in range(1, lot_num+1):
                high_dist[i - 1, 0] = H + c + dist_sign * pow(2, i)
                high_dist[i - 1, 1] = pH / pow(2, i)

            high_dist[lot_num - 1, 1] = high_dist[lot_num - 1, 1] * 2

        # incorporate L into the distribution
        dist = np.copy(high_dist)
        locb = np.where(high_dist[:, 0] == float(L))
        if locb[0].size > 0:
            dist[locb, 1] += (1-pH)
        elif pH < 1:
            dist = np.vstack((dist, [L, 1-pH]))

        dist = dist[np.argsort(dist[:, 0])]

    return dist


In [5]:
def get_pBetter(DistX, DistY, corr, accuracy=10000):
    # Return probability that a value drawn from DistX is strictly larger than one drawn from DistY
    # Input: 2 discrete distributions which are set as matrices of 1st column
    # as outcome and 2nd its probability. DistX and DistY are numpy matrices; correlation between the distributions;
    # level of accuracy in terms of number of samples to take from distributions
    # Output: a list with the estimated probability that X generates value strictly larger than Y, and
    # the probability that Y generates value strictly larger than X

    nXbetter = 0
    nYbetter = 0

    for j in range(1, accuracy+1):
        rndNum = np.random.uniform(size=2)
        sampleX = distSample(DistX[:, 0], DistX[:, 1], rndNum[0])
        if corr == 1:
            sampleY = distSample(DistY[:, 0], DistY[:, 1], rndNum[0])
        elif corr == -1:
            sampleY = distSample(DistY[:, 0], DistY[:, 1], 1-rndNum[0])
        else:
            sampleY = distSample(DistY[:, 0], DistY[:, 1], rndNum[1])

        nXbetter = nXbetter + int(sampleX > sampleY)
        nYbetter = nYbetter + int(sampleY > sampleX)

    pXbetter = nXbetter / accuracy
    pYbetter = nYbetter / accuracy

    return [pXbetter, pYbetter]


In [6]:
def CPC15_isStochasticDom (DistA, DistB):
    # Check if one distribution dominates stochastically the other
    # Input: 2 discrete distributions which are set as matrices of 1st column
    # as outcome and 2nd its probability (DistA and DistB are numpy matrices)
    # Output: pandas data framed with 2 columns:
    # 'is' a logical output, 'which' a char output ('A', 'B', NaN)

    na = DistA.shape[0]
    nb = DistB.shape[0]
    if np.array_equal(DistA, DistB):
        dom = False
        which = None
    else:
        tempa = np.ones(shape=(na, 1))
        tempb = np.ones(shape=(nb, 1))
        for i in range(0, nb):
            sumpa = 0  # DistA(i,2)
            j = 0
            sumpb = np.sum(DistB[0:i + 1, 1])

            while (sumpa != 1) and (j < na) and (sumpa + DistA[j, 1] <= sumpb):
                sumpa += DistA[j, 1]
                if sumpa == sumpb:
                    break
                j += 1

            if j == na:
                j = na - 1
            if i == nb:
                i = nb - 1

            if DistB[i, 0] < DistA[j, 0]:
                tempb[i] = 0
                break

        if np.all(tempb != 0):
            dom = True
            which = 'B'
        else:
            for i in range(0, na):
                sumpb = 0  # DistA(i,2)
                j = 0
                sumpa = np.sum(DistA[0: i+1, 1])

                while (sumpb != 1) and (j < nb) and (sumpb + DistB[j, 1] <= sumpa):
                    sumpb += DistB[j, 1]
                    if sumpa == sumpb:
                        break
                    j += 1

                if j == nb:
                    j = nb - 1
                if i == na:
                    i = na - 1

                if DistA[i, 0] < DistB[j, 0]:
                    tempa[i] = 0
                    break

            if np.all(tempa != 0):
                dom = True
                which = 'A'
            else:
                dom = False
                which = None

    return pd.DataFrame([{'dom': dom, 'which': which}])


In [7]:
def CPC15_BEASTsimulation(DistA, DistB, Amb, Corr):
    # Simulation of the BEAST model.
    #  Input: 2 discrete distributions which are set as matrices of 1st column
    # as outcome and 2nd its probability. DistA and DistB are numpy matrices;
    #  Amb is a number 1 or 0, this is the ambiguous between the A and B.
    #  Corr is thw correlation between A and B, this is a number between -1 to 1.
    # Output: numpy array of zise: (nBlocks, 1)

    SIGMA = 7
    KAPA = 3
    BETA = 2.6
    GAMA = 0.5
    PSI = 0.07
    THETA = 1

    nTrials = 25
    firstFeedback = 6
    nBlocks = 5

    # draw personal traits
    sigma = SIGMA * np.random.uniform(size=1)
    kapa = np.random.choice(range(1, KAPA+1), 1)
    beta = BETA * np.random.uniform(size=1)
    gama = GAMA * np.random.uniform(size=1)
    psi = PSI * np.random.uniform(size=1)
    theta = THETA * np.random.uniform(size=1)

    ObsPay = np.zeros(shape=(nTrials - firstFeedback + 1, 2))  # observed outcomes in A (col1) and B (col2)
    Decision = np.empty(shape=(nTrials, 1))
    simPred = np.empty(shape=(nBlocks, 1))

    # Useful variables
    nA = DistA.shape[0]  # num outcomes in A
    nB = DistB.shape[0]  # num outcomes in B

    if Amb == 1:
        ambiguous = True
    else:
        ambiguous = False

    nfeed = 0  # "t"; number of outcomes with feedback so far
    pBias = np.array([beta / (beta + 1 + pow(nfeed, theta))])
    MinA = DistA[0, 0]
    MinB = DistB[0, 0]
    MaxOutcome = np.maximum(DistA[nA - 1, 0], DistB[nB - 1, 0])
    SignMax = np.sign(MaxOutcome)

    if MinA == MinB:
        RatioMin = 1
    elif np.sign(MinA) == np.sign(MinB):
        RatioMin = min(abs(MinA), abs(MinB)) / max(abs(MinA), abs(MinB))
    else:
        RatioMin = 0

    Range = MaxOutcome - min(MinA, MinB)
    trivial = CPC15_isStochasticDom(DistA, DistB)
    BEVa = np.matrix.dot(DistA[:, 0], DistA[:, 1])
    if ambiguous:
        UEVb = np.matrix.dot(DistB[:, 0], np.repeat([1 / nB], nB))
        BEVb = (1-psi) * (UEVb+BEVa) / 2 + psi * MinB
        pEstB = np.repeat([float(nB)], 1)  # estimation of probabilties in Amb
        t_SPminb = (BEVb - np.mean(DistB[1:nB+1, 0])) / (MinB - np.mean(DistB[1:nB+1, 0]))
        if t_SPminb < 0:
            pEstB[0] = 0
        elif t_SPminb > 1:
            pEstB[0] = 1
        else:
            pEstB[0] = t_SPminb

        # Add nb-1 rows to pEstB:
        pEstB = np.append(pEstB, np.repeat((1 - pEstB[0]) / (nB - 1), nB-1))

    else:
        pEstB = DistB[:, 1]
        BEVb = np.matrix.dot(DistB[:, 0], pEstB)

    # simulation of decisions
    for trial in range(nTrials):
        STa = 0
        STb = 0
        # mental simulations
        for s in range(1, kapa[0]+1):
            rndNum = np.random.uniform(size=2)
            if rndNum[0] > pBias[nfeed]:  # Unbiased technique
                if nfeed == 0:
                    outcomeA = distSample(DistA[:, 0], DistA[:, 1], rndNum[1])
                    outcomeB = distSample(DistB[:, 0], pEstB, rndNum[1])
                else:
                    uniprobs = np.repeat([1 / nfeed], nfeed)
                    outcomeA = distSample(ObsPay[0:nfeed, 0], uniprobs, rndNum[1])
                    outcomeB = distSample(ObsPay[0:nfeed, 1], uniprobs, rndNum[1])

            elif rndNum[0] > (2 / 3) * pBias[nfeed]:  # uniform
                outcomeA = distSample(DistA[:, 0], np.repeat([1 / nA], nA), rndNum[1])
                outcomeB = distSample(DistB[:, 0], np.repeat([1 / nB], nB), rndNum[1])

            elif rndNum[0] > (1 / 3) * pBias[nfeed]:  # contingent pessimism
                if SignMax > 0 and RatioMin < gama:
                    outcomeA = MinA
                    outcomeB = MinB
                else:
                    outcomeA = distSample(DistA[:, 0], np.repeat([1 / nA], nA), rndNum[1])
                    outcomeB = distSample(DistB[:, 0], np.repeat([1 / nB], nB), rndNum[1])

            else:  # Sign
                if nfeed == 0:
                    outcomeA = Range * distSample(np.sign(DistA[:, 0]), DistA[:, 1], rndNum[1])
                    outcomeB = Range * distSample(np.sign(DistB[:, 0]), pEstB, rndNum[1])
                else:
                    uniprobs = np.repeat([1 / nfeed], nfeed)
                    outcomeA = Range * distSample(np.sign(ObsPay[0:nfeed, 0]), uniprobs, rndNum[1])
                    outcomeB = Range * distSample(np.sign(ObsPay[0:nfeed, 1]), uniprobs, rndNum[1])

            STa = STa + outcomeA
            STb = STb + outcomeB

        STa = STa / kapa
        STb = STb / kapa

        # error term
        if trivial['dom'][0]:
            error = 0
        else:
            error = sigma * np.random.normal(size=1)  # positive values contribute to attraction to A

        # decision
        Decision[trial] = (BEVa - BEVb) + (STa - STb) + error < 0
        if (BEVa - BEVb) + (STa - STb) + error == 0:
            Decision[trial] = np.random.choice(range(1, 3), size=1, replace=False) - 1

        if trial >= firstFeedback - 1:
            #  got feedback
            nfeed += 1
            pBias = np.append(pBias, beta / (beta + 1 + pow(nfeed, theta)))
            rndNumObs = np.random.uniform(size=1)
            ObsPay[nfeed - 1, 0] = distSample(DistA[:, 0], DistA[:, 1], rndNumObs)  # draw outcome from A
            if Corr == 1:
                ObsPay[nfeed - 1, 1] = distSample(DistB[:, 0], DistB[:, 1], rndNumObs)
            elif Corr == -1:
                ObsPay[nfeed - 1, 1] = distSample(DistB[:, 0], DistB[:, 1], 1-rndNumObs)
            else:
                # draw outcome from B
                ObsPay[nfeed - 1, 1] = distSample(DistB[:, 0], DistB[:, 1], np.random.uniform(size=1))
            if ambiguous:
                BEVb = (1 - 1 / (nTrials-firstFeedback+1)) * BEVb + 1 / (nTrials-firstFeedback+1) * ObsPay[nfeed - 1, 1]

    # compute B-rates for this simulation
    blockSize = nTrials / nBlocks
    for b in range(1, nBlocks+1):
        simPred[b-1] = np.mean(Decision[int(((b - 1) * blockSize + 1)-1):int(b * blockSize)])

    return simPred


In [8]:
def CPC15_BEASTpred(Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr):
    # Prediction of (the original) BEAST model for one problem
    # Input: for a and b: high outcome (Ha/ Hb: int), its probability (pHa/ pHb: double), low outcome
    #  (La/ Lb: int), the shape of the lottery (LotShapeA/ LotShapeB that can be:'-'/'Symm'/'L-skew'/'R-skew' only),
    #  the number of outcomes in the lottery (lot_numA/ LotNumB: int),
    #  Amb indicates if B is ambiguous (=1) or not (=0).
    #  Corr is the correlation between A and B, this is a number between -1 to 1.
    # Output: is the prediction of the BEAST model: this is a numpy of size (5,1)

    Prediction = np.repeat([0], 5)
    Prediction.shape = (5, 1)

    # get both options' distributions
    DistA = CPC18_getDist(Ha, pHa, La, LotShapeA, LotNumA)
    DistB = CPC18_getDist(Hb, pHb, Lb, LotShapeB, LotNumB)

    # run model simulation nSims times
    nSims = 4000
    for sim in range(0, nSims):
        simPred = CPC15_BEASTsimulation(DistA, DistB, Amb, Corr)
        Prediction = np.add(Prediction, (1 / nSims) * simPred)

    return Prediction

In [9]:
def get_PF_Features(Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr):
    # Finds the values of the engineered features that are part of Psychological Forest
    # Gets as input the parameters defining the choice problem in CPC18 and returns
    # as output a pandas data frame with this problem's features

    # Compute "naive" and "psychological" features as per Plonsky, Erev, Hazan, and Tennenholtz, 2017
    DistA = CPC18_getDist(Ha, pHa, La, LotShapeA, LotNumA)
    DistB = CPC18_getDist(Hb, pHb, Lb, LotShapeB, LotNumB)
    diffEV = (np.matrix.dot(DistB[:, 0], DistB[:, 1]) - np.matrix.dot(DistA[:, 0], DistA[:, 1]))
    diffSDs = (getSD(DistB[:, 0], DistB[:, 1]) - getSD(DistA[:, 0], DistA[:, 1]))
    MinA = DistA[0, 0]
    MinB = DistB[0, 0]
    diffMins = MinB - MinA
    nA = DistA.shape[0]  # num outcomes in A
    nB = DistB.shape[0]  # num outcomes in B
    MaxA = DistA[nA - 1, 0]
    MaxB = DistB[nB - 1, 0]
    diffMaxs = MaxB - MaxA

    diffUV = (np.matrix.dot(DistB[:, 0], np.repeat([1 / nB], nB))) - (np.matrix.dot(DistA[:, 0], np.repeat([1 / nA], nA)))
    if Amb == 1:
        ambiguous = True
    else:
        ambiguous = False

    MaxOutcome = max(MaxA, MaxB)
    SignMax = np.sign(MaxOutcome)
    if MinA == MinB:
        RatioMin = 1
    elif np.sign(MinA) == np.sign(MinB):
        RatioMin = min(abs(MinA), abs(MinB)) / max(abs(MinA), abs(MinB))
    else:
        RatioMin = 0

    Range = MaxOutcome - min(MinA, MinB)
    diffSignEV = (Range * np.matrix.dot(np.sign(DistB[:, 0]), DistB[:, 1]) -
                  Range * np.matrix.dot(np.sign(DistA[:, 0]), DistA[:, 1]))
    trivial = CPC15_isStochasticDom(DistA, DistB)
    whchdom = trivial['which'][0]
    Dom = 0
    if trivial['dom'][0] and whchdom == 'A':
        Dom = -1
    if trivial['dom'][0] and whchdom == 'B':
        Dom = 1
    BEVa = np.matrix.dot(DistA[:, 0], DistA[:, 1])
    if ambiguous:
        UEVb = np.matrix.dot(DistB[:, 0], np.repeat(1 / nB, nB))
        BEVb = (UEVb + BEVa + MinB) / 3
        pEstB = np.repeat([float(nB)], 1)  # estimation of probabilties in Amb
        t_SPminb = (BEVb - np.mean(DistB[1:nB + 1, 0])) / (MinB - np.mean(DistB[1:nB + 1, 0]))
        if t_SPminb < 0:
            pEstB[0] = 0
        elif t_SPminb > 1:
            pEstB[0] = 1
        else:
            pEstB[0] = t_SPminb
        pEstB = np.append(pEstB, np.repeat([(1 - pEstB[0]) / (nB - 1)], nB - 1))
    else:
        pEstB = DistB[:, 1]
        BEVb = np.matrix.dot(DistB[:, 0], pEstB)

    diffBEV0 = (BEVb - BEVa)
    BEVfb = (BEVb + (np.matrix.dot(DistB[:, 0], DistB[:, 1]))) / 2
    diffBEVfb = (BEVfb - BEVa)

    sampleDistB = np.column_stack((DistB[:, 0], pEstB))
    probsBetter = get_pBetter(DistA, sampleDistB, corr=1)
    pAbetter = probsBetter[0]
    pBbetter = probsBetter[1]
    pBbet_Unbiased1 = pBbetter - pAbetter

    sampleUniDistA = np.column_stack((DistA[:, 0], np.repeat([1 / nA], nA)))
    sampleUniDistB = np.column_stack((DistB[:, 0], np.repeat([1 / nB], nB)))
    probsBetterUni = get_pBetter(sampleUniDistA, sampleUniDistB, corr=1)
    pBbet_Uniform = probsBetterUni[1] - probsBetterUni[0]

    sampleSignA = np.copy(DistA)
    sampleSignA[:, 0] = np.sign(sampleSignA[:, 0])
    sampleSignB = np.column_stack((np.sign(DistB[:, 0]), pEstB))
    probsBetterSign = get_pBetter(sampleSignA, sampleSignB, corr=1)
    pBbet_Sign1 = probsBetterSign[1] - probsBetterSign[0]
    sampleSignBFB = np.column_stack((np.sign(DistB[:, 0]), DistB[:, 1]))
    if Corr == 1:
        probsBetter = get_pBetter(DistA, DistB, corr=1)
        probsBetterSign = get_pBetter(sampleSignA, sampleSignBFB, corr=1)
    elif Corr == -1:
        probsBetter = get_pBetter(DistA, DistB, corr=-1)
        probsBetterSign = get_pBetter(sampleSignA, sampleSignBFB, corr=-1)
    else:
        probsBetter = get_pBetter(DistA, DistB, corr=0)
        probsBetterSign = get_pBetter(sampleSignA, sampleSignBFB, corr=0)

    pBbet_UnbiasedFB = probsBetter[1] - probsBetter[0]
    pBbet_SignFB = probsBetterSign[1] - probsBetterSign[0]

    # convert lot shape: '-'/'Symm'/'L-skew'/'R-skew' to 4 different features for the RF model
    lot_shape_listA = lot_shape_convert(LotShapeA)
    lot_shape_listB = lot_shape_convert(LotShapeB)

    # create features data frame
    feats_labels = ('Ha', 'pHa', 'La', 'lot_shape__A', 'lot_shape_symm_A', 'lot_shape_L_A', 'lot_shape_R_A', 'LotNumA',
                    'Hb', 'pHb', 'Lb', 'lot_shape__B', 'lot_shape_symm_B', 'lot_shape_L_B', 'lot_shape_R_B', 'LotNumB',
                    'Amb', 'Corr', 'diffEV', 'diffSDs', 'diffMins', 'diffMaxs', 'diffUV', 'RatioMin', 'SignMax',
                    'pBbet_Unbiased1', 'pBbet_UnbiasedFB', 'pBbet_Uniform', 'pBbet_Sign1', 'pBbet_SignFB', 'Dom',
                    'diffBEV0', 'diffBEVfb', 'diffSignEV')
    data_lists = [[Ha, pHa, La], lot_shape_listA, [LotNumA, Hb, pHb, Lb], lot_shape_listB, [LotNumB, Amb, Corr,
                             diffEV, diffSDs, diffMins, diffMaxs, diffUV, RatioMin, SignMax, pBbet_Unbiased1,
                             pBbet_UnbiasedFB, pBbet_Uniform, pBbet_Sign1, pBbet_SignFB, Dom, diffBEV0,
                             diffBEVfb, diffSignEV]]
    features_data = [item for sublist in data_lists for item in sublist]
    tmpFeats = pd.DataFrame(features_data, index=feats_labels).T

    # duplicate features data frame as per number of blocks
    Feats = pd.concat([tmpFeats] * 5)

    # get BEAST model prediction as feature
    beastPs = CPC15_BEASTpred(Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr)
    Feats['BEASTpred'] = beastPs

    Feats['block'] = np.arange(1, 6)
    Feats['Feedback'] = 1
    Feats.loc[Feats['block'] == 1, 'Feedback'] = 0

    return Feats


# To compute the distribution's standard deviation
def getSD(vals, probs):
    m = np.matrix.dot(vals, probs.T)
    sqds = np.power((vals - m), 2)
    var = np.matrix.dot(probs, sqds.T)
    return math.sqrt(var)


# Convert lot shape feautre to vector of 4 features
def lot_shape_convert(lot_shape):
    return {
        '-': [1, 0, 0, 0],
        'Symm': [0, 1, 0, 0],
        'L-skew': [0, 0, 1, 0],
        'R-skew': [0, 0, 0, 1],
    }[lot_shape]


## New Model:
I have used an MLPRegressor here.

In [10]:
def CPC18_PF_pred(train_data, Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr):
    Feats = get_PF_Features(Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr)
    n_runs = 10
    prediction = np.repeat([0], 5)
    prediction.shape = (1, 5)
    for run in range(n_runs):

        x_train = train_data.iloc[:, 1:38]
        y_train = train_data['B_rate']
        mlp_model =  MLPRegressor(max_iter=500, hidden_layer_sizes=[200, 275, 100])
        mlp_model.fit(X=x_train, y=y_train)

        pred = mlp_model.predict(Feats)
        prediction = np.add(prediction, (1 / n_runs) * pred)

    return prediction, mlp_model

In [1]:
####################################################
### Section B: Please do not change this section ###
####################################################
# load problems to predict (in this example, the estimation set problems)
Data = pd.read_csv('CPC18_EstSet.csv')
train_data = pd.read_csv('TrainData.csv')
# useful variables
nProblems = Data.shape[0]
PredictedAll = np.zeros(shape=(nProblems, 5))
### End of Section A ###
#################################################################
### Section C: Please change only lines 41-47 in this section ###
#################################################################
for prob in range(nProblems):
    # read problem's parameters
    Ha = Data['Ha'][prob]
    pHa = Data['pHa'][prob]
    La = Data['La'][prob]
    LotShapeA = Data['LotShapeA'][prob]
    LotNumA = Data['LotNumA'][prob]
    Hb = Data['Hb'][prob]
    pHb = Data['pHb'][prob]
    Lb = Data['Lb'][prob]
    LotShapeB = Data['LotShapeB'][prob]
    LotNumB = Data['LotNumB'][prob]
    Amb = Data['Amb'][prob]
    Corr = Data['Corr'][prob]
    # new model
    Prediction, model = CPC18_PF_pred(train_data, Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb,
                               Corr)
    # end of example
    PredictedAll[prob, :] = Prediction
    # for verbose progression
    print('{}: Finish problem number: {}'.format((time.asctime(time.localtime(time.time()))), prob + 1))
    # pd.DataFrame(model.loss_curve_).plot()
### End of Section C ###

In [46]:
####################################################
### Section D: Please do not change this section ###
####################################################
# compute MSE
ObservedAll = Data[['B.1', 'B.2', 'B.3', 'B.4', 'B.5']]
probMSEs = 100 * ((PredictedAll - ObservedAll) ** 2).mean(axis=1)
totalMSE = np.mean(probMSEs)
print('MSE over the {} problems: {}'.format(nProblems, totalMSE))
# for keeping the predicted choice rates
np.savetxt("outputAll_MLP.csv", PredictedAll, delimiter=",", header="B1,B2,B3,B4,B5", fmt='%.4e')
### End of Section D ###


MSE over the 60 problems: 9.034410759679913


### Modification ( same modification as in the baseline)

In [11]:
def CPC18_PF_pred_new(train_data, Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr, mlp_model):
    Feats = get_PF_Features(Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr)
    
    n_runs = 10
    prediction = np.repeat([0], 5)
    prediction.shape = (1, 5)
    for run in range(n_runs):
        x_train = train_data.iloc[:, 1:38]
        y_train = train_data['B_rate']
        mlp_model.fit(X=x_train, y=y_train)
        pred = mlp_model.predict(Feats)
        # errors.append(log_loss(y_train, pred))
        prediction = np.add(prediction, (1 / n_runs) * pred)

    return prediction, mlp_model

In [12]:
if __name__ == '__main__':
    
    Data = pd.read_csv('CPC18_EstSet.csv')
    train_data = pd.read_csv('TrainData.csv')
    nProblems = Data.shape[0]
    PredictedAll = np.zeros(shape=(nProblems, 5))
    
    # errors = []
    
    mlp_model = MLPRegressor(max_iter=500, hidden_layer_sizes=[200, 275, 100], warm_start=True)
    for prob in range(nProblems):
        # read problem's parameters
        Ha = Data['Ha'][prob]
        pHa = Data['pHa'][prob]
        La = Data['La'][prob]
        LotShapeA = Data['LotShapeA'][prob]
        LotNumA = Data['LotNumA'][prob]
        Hb = Data['Hb'][prob]
        pHb = Data['pHb'][prob]
        Lb = Data['Lb'][prob]
        LotShapeB = Data['LotShapeB'][prob]
        LotNumB = Data['LotNumB'][prob]
        Amb = Data['Amb'][prob]
        Corr = Data['Corr'][prob]


        Prediction, mlp_model = CPC18_PF_pred_new(train_data, Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb,
                                   Corr, mlp_model)

        PredictedAll[prob, :] = Prediction
        print('{}: Finish problem number: {}'.format((time.asctime(time.localtime(time.time()))), prob + 1))

  

ObservedAll = Data[['B.1', 'B.2', 'B.3', 'B.4', 'B.5']]
probMSEs = 100 * ((PredictedAll - ObservedAll) ** 2).mean(axis=1)
totalMSE = np.mean(probMSEs)
print('MSE over the {} problems: {}'.format(nProblems, totalMSE))

Sat May  6 02:46:36 2023: Finish problem number: 1
Sat May  6 02:46:49 2023: Finish problem number: 2
Sat May  6 02:47:00 2023: Finish problem number: 3
Sat May  6 02:47:12 2023: Finish problem number: 4
Sat May  6 02:47:24 2023: Finish problem number: 5
Sat May  6 02:47:36 2023: Finish problem number: 6
Sat May  6 02:47:48 2023: Finish problem number: 7
Sat May  6 02:48:01 2023: Finish problem number: 8
Sat May  6 02:48:13 2023: Finish problem number: 9
Sat May  6 02:48:27 2023: Finish problem number: 10
Sat May  6 02:48:41 2023: Finish problem number: 11
Sat May  6 02:48:53 2023: Finish problem number: 12
Sat May  6 02:49:05 2023: Finish problem number: 13
Sat May  6 02:49:16 2023: Finish problem number: 14
Sat May  6 02:49:28 2023: Finish problem number: 15
Sat May  6 02:49:39 2023: Finish problem number: 16
Sat May  6 02:49:50 2023: Finish problem number: 17
Sat May  6 02:50:03 2023: Finish problem number: 18
Sat May  6 02:50:14 2023: Finish problem number: 19
Sat May  6 02:50:26 2

The Modification improves the MSE but it's still not as good as the baseline.


#### I further increase the number of hidden layers and the number of units in each hidden layer.

In [15]:
if __name__ == '__main__':
    
    Data = pd.read_csv('CPC18_EstSet.csv')
    train_data = pd.read_csv('TrainData.csv')
    nProblems = Data.shape[0]
    PredictedAll = np.zeros(shape=(nProblems, 5))
    
    # errors = []
    
    mlp_model = MLPRegressor(max_iter=1000, hidden_layer_sizes=[1024, 512, 256, 128, 64, 32], warm_start=True)
    for prob in range(nProblems):
        # read problem's parameters
        Ha = Data['Ha'][prob]
        pHa = Data['pHa'][prob]
        La = Data['La'][prob]
        LotShapeA = Data['LotShapeA'][prob]
        LotNumA = Data['LotNumA'][prob]
        Hb = Data['Hb'][prob]
        pHb = Data['pHb'][prob]
        Lb = Data['Lb'][prob]
        LotShapeB = Data['LotShapeB'][prob]
        LotNumB = Data['LotNumB'][prob]
        Amb = Data['Amb'][prob]
        Corr = Data['Corr'][prob]


        Prediction, mlp_model = CPC18_PF_pred_new(train_data, Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb,
                                   Corr, mlp_model)

        PredictedAll[prob, :] = Prediction
        print('{}: Finish problem number: {}'.format((time.asctime(time.localtime(time.time()))), prob + 1))

  

ObservedAll = Data[['B.1', 'B.2', 'B.3', 'B.4', 'B.5']]
probMSEs = 100 * ((PredictedAll - ObservedAll) ** 2).mean(axis=1)
totalMSE = np.mean(probMSEs)
print('MSE over the {} problems: {}'.format(nProblems, totalMSE))

Sat May  6 03:37:40 2023: Finish problem number: 1
Sat May  6 03:37:55 2023: Finish problem number: 2
Sat May  6 03:38:09 2023: Finish problem number: 3
Sat May  6 03:38:22 2023: Finish problem number: 4
Sat May  6 03:38:35 2023: Finish problem number: 5
Sat May  6 03:38:49 2023: Finish problem number: 6
Sat May  6 03:39:02 2023: Finish problem number: 7
Sat May  6 03:39:15 2023: Finish problem number: 8
Sat May  6 03:39:27 2023: Finish problem number: 9
Sat May  6 03:39:40 2023: Finish problem number: 10
Sat May  6 03:39:54 2023: Finish problem number: 11
Sat May  6 03:40:09 2023: Finish problem number: 12
Sat May  6 03:40:23 2023: Finish problem number: 13
Sat May  6 03:40:38 2023: Finish problem number: 14
Sat May  6 03:40:49 2023: Finish problem number: 15
Sat May  6 03:41:02 2023: Finish problem number: 16
Sat May  6 03:41:16 2023: Finish problem number: 17
Sat May  6 03:41:31 2023: Finish problem number: 18
Sat May  6 03:41:46 2023: Finish problem number: 19
Sat May  6 03:42:02 2

#### The MSE improves even further.